# *Gendered Pronoun Resolution*

En el análisis de texto natural, existen oraciones complejas de entender incluso para las personas. Uno de los casos más conflictivos son los pronombres ambiguos. En 2018, se publicó un dataset junto con el paper [A Balanced Corpus of Gendered Ambiguous Pronouns](https://arxiv.org/pdf/1810.05201.pdf), donde se proponen un conjunto de textos con pronombres ambiguos con género.

El objetivo de este dataset es encontrar el nombre en el texto al que el pronombre ambiguo hace referencia.

Para ello se nos da un dataset con los siguientes campos:

* `ID`: Identificador de la frase.
* `Text`: Texto en fromato string.
* `Pronoun`: string con el pronombre ambiguo.
* `Pronoun-offset`: índice del carácter donde empieza el pronombre dentro del texto.
* `A`: string con el primer nombre candidato a hacer referencia por el pronombre.
* `A-offset`: índice del carácter donde empieza el nombre A dentro del texto.
* `A-coref`: boleano indicando si el pronombre hace referencia al nombre A.
* `B`: string con el segundo nombre candidato a hacer referencia por el pronombre.
* `B-offset`: índice del carácter donde empieza el nombre B dentro del texto.
* `B-coref`: boleano indicando si el pronombre hace referencia al nombre B.
* `URL`: web de donde se ha sacado el fragmento de texto.

## Objetivo:

Hacer una predicción de a cuál de los dos nombres marcados en cada frase hace referencia el pronombre seleccionado usando **dos modelos distintos** de PNL siguiendo el formato descrito a continuación:

* **MODELO 1**: Puede ser **cualquier modelo visto en los seminarios de PLN o en otras asignaturas**, como: Count vectorizer, HMM, Structured Perceptron, RNN, Logistic Regressor, XGBoost, etc...

    * Justificar el porqué del modelo elegido.
    * Entrenar el modelo.
    * Dar una accuracy del modelo.
    * Interpretar y explicar los resultados del modelo.
 

* **MODELO 2**: Debe ser un modelo **basado en Transformers** que incorpore el concepto de ***attention***.

    * Justificar el porqué del modelo elegido.
    * Entrenar el modelo.
    * Dar una accuracy del modelo.
    * Interpretar y explicar los resultados del modelo.    
    
    





## Libraries

In [368]:
import os
import sys

import pandas as pd
import numpy as np

import re
import contractions
import string 
import colorama
from colorama import Fore

from sklearn.metrics import classification_report


## Load data

In [449]:
print(os.listdir("./input/gap-coreference-master"))

['gap-development.tsv', 'gap-test.tsv', 'gap-validation.tsv']


In [450]:
DATA_ROOT = './input/'
GAP_DATA_FOLDER = os.path.join(DATA_ROOT, 'gap-coreference-master')
GAP_DATA_FOLDER

'./input/gap-coreference-master'

In [451]:
train_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-development.tsv')
test_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-test.tsv')
val_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-validation.tsv')

train_df = pd.read_csv(train_df_path, sep='\t') # train_df
test_df = pd.read_csv(test_df_path, sep='\t')
val_df = pd.read_csv(val_df_path, sep='\t')


## EDA

In [6]:
train_df.head()

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL
0,development-1,Zoe Telford -- played the police officer girlf...,her,274,Cheryl Cassidy,191,True,Pauline,207,False,http://en.wikipedia.org/wiki/List_of_Teachers_...
1,development-2,"He grew up in Evanston, Illinois the second ol...",His,284,MacKenzie,228,True,Bernard Leach,251,False,http://en.wikipedia.org/wiki/Warren_MacKenzie
2,development-3,"He had been reelected to Congress, but resigne...",his,265,Angeloz,173,False,De la Sota,246,True,http://en.wikipedia.org/wiki/Jos%C3%A9_Manuel_...
3,development-4,The current members of Crime have also perform...,his,321,Hell,174,False,Henry Rosenthal,336,True,http://en.wikipedia.org/wiki/Crime_(band)
4,development-5,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,Kitty Oppenheimer,219,False,Rivera,294,True,http://en.wikipedia.org/wiki/Jessica_Rivera


In [80]:
# check null values
train_df.isnull().sum()

Text              0
Pronoun           0
Pronoun-offset    0
A                 0
A-offset          0
B                 0
B-offset          0
A-coref           0
B-coref           0
dtype: int64

- Observamos que no hay valores nulos en el dataset. 

In [64]:
# describe the data 
train_df.describe()

,Pronoun-offset,A-offset,B-offset
count,2000.000000,2000.00000,2000.000000
mean,324.963500,239.77800,300.535500
std,98.788591,111.15768,113.226357
min,3.000000,0.00000,16.000000
25%,274.000000,179.75000,237.000000
50%,316.000000,239.00000,294.000000
75%,370.000000,301.25000,358.000000
max,1135.000000,971.00000,1098.000000


In [74]:
# Data imbalance
print(train_df["A-coref"].value_counts())

print(f"\n{train_df['B-coref'].value_counts()}")

A-coref
False    1126
True      874
Name: count, dtype: int64

B-coref
False    1075
True      925
Name: count, dtype: int64


- El desbalanceo de clases no es extremadamente pronunciado. 

In [76]:
# full naw text
raw_text = train_df["Text"]
raw_text

0       Zoe Telford -- played the police officer girlf...
1       He grew up in Evanston, Illinois the second ol...
2       He had been reelected to Congress, but resigne...
3       The current members of Crime have also perform...
4       Her Santa Fe Opera debut in 2005 was as Nuria ...
                              ...                        
1995    Faye's third husband, Paul Resnick, reported t...
1996    The plot of the film focuses on the life of a ...
1997    Grant played the part in Trevor Nunn's movie a...
1998    The fashion house specialised in hand-printed ...
1999    Watkins was a close friend of Hess' first wife...
Name: Text, Length: 2000, dtype: object

In [7]:
# Para visualizar las frases
def gap_printer2(data_df_row):
        
    text   = data_df_row["Text"]
    word_A = data_df_row["A"]
    word_B = data_df_row["B"]
    
    pronoun       = data_df_row["Pronoun"]
    pronoun_begin = data_df_row["Pronoun-offset"]
    pronoun_end   = pronoun_begin + len(pronoun)
    
    word_A_begin = data_df_row["A-offset"]
    word_A_end   = data_df_row["A-offset"] + len(word_A)
    word_B_begin = data_df_row["B-offset"]
    word_B_end   = data_df_row["B-offset"] + len(word_B)
    
    text_c = text.replace(word_A, " {} ")
    text_c = text.replace(word_B, " {} ")
    text_c = text.replace(pronoun, " {} ")
    
    word_boundaries = np.sort([word_A_begin, word_A_end, pronoun_begin, pronoun_end, word_B_begin, word_B_end])
    word_boundaries = list(zip(word_boundaries[::2], word_boundaries[1::2]))
    
    P1 = [0,word_boundaries[0][0]]
    P2 = [word_boundaries[0][1],word_boundaries[1][0]]
    P3 = [word_boundaries[1][1],word_boundaries[2][0]]
    P4 = [word_boundaries[2][1],len(text)]

    text_f = text[P1[0]:P1[1]] + "{}" + text[P2[0]:P2[1]] +  "{}" + text[P3[0]:P3[1]] + "{}" + text[P4[0]:P4[1]]
 
    print(text_f.format( Fore.BLUE  + text[word_boundaries[0][0]:word_boundaries[0][1]]  + Fore.BLACK,
                         Fore.BLUE  + text[word_boundaries[1][0]:word_boundaries[1][1]] + Fore.BLACK,
                         Fore.BLUE  + text[word_boundaries[2][0]:word_boundaries[2][1]]  + Fore.BLACK))

In [5]:
gap_printer2(train_df.loc[3])

The current members of Crime have also performed in San Francisco under the band name ''Remote Viewers``. Strike has published two works of fiction in recent years: Ports of Hell, which is listed in the Rock and Roll Hall of Fame Library, and A Loud Humming Sound Came from Above. Rank has produced numerous films (under his real name, Henry Rosenthal) including the hit The Devil and Daniel Johnston.


- El pronombre que tenemos que identificar es `his`. 
- Tenemos 2 posibles nombres a los que hace referencia: `Hell` y `Henry Rosenthal`.


In [6]:
# extraemos la URL del segundo elemento del dataset
url = train_df["URL"][1]
url

'http://en.wikipedia.org/wiki/Warren_MacKenzie'

In [8]:
# extraemos el segundo texto del dataset
text = train_df["Text"][1]
text

'He grew up in Evanston, Illinois the second oldest of five children including his brothers, Fred and Gordon and sisters, Marge (Peppy) and Marilyn. His high school days were spent at New Trier High School in Winnetka, Illinois. MacKenzie studied with Bernard Leach from 1949 to 1952. His simple, wheel-thrown functional pottery is heavily influenced by the oriental aesthetic of Shoji Hamada and Kanjiro Kawai.'

In [9]:
# extraemos el primer pronombre del segundo elemento del dataset
pronoun = train_df["Pronoun"][1]
pronoun

'His'

In [10]:
# extraemos el offset del primer pronombre del segundo elemento del dataset
pronoun_offset = train_df["Pronoun-offset"][1]
pronoun_offset

284

In [11]:
# visualizamos de otra forma el pronombre. 
# A partir de la variable Text, si seleccionamos únicamente la parte que empieza
# en el pronoun offset y acaba en el pronoun offset + el tamaño de ese pronombre, 
# lo que extraemos del texto es el pronombre. 
text[pronoun_offset:pronoun_offset+len(pronoun)]

'His'

In [12]:
# Extraemos el nombre A y el A-offset
A = train_df["A"][1]
A_offset = train_df["A-offset"][1]
A, A_offset

('MacKenzie', 228)

In [13]:
# nombre A
text[A_offset:A_offset+len(A)]

'MacKenzie'

In [14]:
# nombre B y B-offset
B = train_df["B"][1]
B_offset = train_df["B-offset"][1]
B, B_offset

('Bernard Leach', 251)

In [15]:
# nombre B 
text[B_offset:B_offset+len(B)]

'Bernard Leach'

## 1st Variable Selection

Nos quedamos únicamente con las variables más interesantes: la URL no nos hace falta para un problema de procesado de texto.

In [443]:
def select_imp_features(df):
    imp_features =["Text", "Pronoun", "Pronoun-offset", "A", "A-offset", "B", "B-offset"]
    target_col = ["A-coref", "B-coref"]
    df = df[imp_features+target_col]    

In [ ]:
train_df = select_imp_features(train_df)

## Text cleaning 

### To lower case 

In [224]:
# To lower case
def lower_case(df):
    df["text_clean"] = df["Text"].apply(lambda x: x.lower())
    df["Pronoun"] = df["Pronoun"].apply(lambda x: x.lower())
    df["A"] = df["A"].apply(lambda x: x.lower())
    df["B"] = df["B"].apply(lambda x: x.lower())

    return df

In [11]:
train_df_clean = lower_case(train_df)
train_df_clean.head()

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,A-coref,B-coref,text_clean
0,Zoe Telford -- played the police officer girlf...,her,274,cheryl cassidy,191,pauline,207,True,False,zoe telford -- played the police officer girlf...
1,"He grew up in Evanston, Illinois the second ol...",His,284,mackenzie,228,bernard leach,251,True,False,"he grew up in evanston, illinois the second ol..."
2,"He had been reelected to Congress, but resigne...",his,265,angeloz,173,de la sota,246,False,True,"he had been reelected to congress, but resigne..."
3,The current members of Crime have also perform...,his,321,hell,174,henry rosenthal,336,False,True,the current members of crime have also perform...
4,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,kitty oppenheimer,219,rivera,294,False,True,her santa fe opera debut in 2005 was as nuria ...


### Expand the contractions: Contraction Mapping

In [12]:
def expand_contractions(df):
    df["text_clean"] = df["text_clean"].apply(lambda x: contractions.fix(x))
    return df

In [13]:
train_df_clean = expand_contractions(train_df_clean)
# double check
print(train_df_clean["Text"][0])
print(train_df_clean["text_clean"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["text_clean"][100])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
zoe telford -- played the police officer girlfriend of simon, maggie. dumped by simon in the final episode of series 1, after he slept with jenny, and is not seen again. phoebe thomas played cheryl cassidy, pauline's friend and also a year 11 pupil in simon's class. dumped her boyfriend following simon's advice after he would not have sex with her but later realised this was due to him catching crabs off her friend pauline.
Re-elected in the 2007 election, she was re-named the Minister of International Relations, La Francophonie and for the Estrie Region as well as t

### Remove non-characters and URLs 

In [14]:
def remove_non_ascii_characters(df, col='text_clean'):
    df[col] = df[col].apply(lambda text: re.sub(r'[^\x00-\x7f]', r'', text)) # get rid of non-characters and whitespace
    return df

In [15]:
train_df_clean = remove_non_ascii_characters(train_df_clean)

# double check
print(train_df_clean["Text"][0])
print(train_df_clean["text_clean"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["text_clean"][100])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
zoe telford -- played the police officer girlfriend of simon, maggie. dumped by simon in the final episode of series 1, after he slept with jenny, and is not seen again. phoebe thomas played cheryl cassidy, pauline's friend and also a year 11 pupil in simon's class. dumped her boyfriend following simon's advice after he would not have sex with her but later realised this was due to him catching crabs off her friend pauline.
Re-elected in the 2007 election, she was re-named the Minister of International Relations, La Francophonie and for the Estrie Region as well as t

### Remove punctuations

In [16]:
def remove_punctuations(df, col='text_clean'):
    """
     - str.maketrans('', '', string.punctuation) crea un traductor utilizando maketrans 
       que mapea los caracteres de puntuación a None, es decir, los elimina.
     - string.punctuation es una cadena predefinida en el módulo string que contiene todos 
       los caracteres de puntuación.
     - text.translate(translator) aplica el traductor al texto, reemplazando las puntuaciones 
       con caracteres vacíos, lo que efectivamente las elimina.
    """
    df[col] = df[col].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
    # return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
    return df

In [17]:
train_df_clean = remove_punctuations(train_df_clean)
# double check
print(train_df_clean["Text"][0])
print(train_df_clean["text_clean"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["text_clean"][100])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
zoe telford  played the police officer girlfriend of simon maggie dumped by simon in the final episode of series 1 after he slept with jenny and is not seen again phoebe thomas played cheryl cassidy paulines friend and also a year 11 pupil in simons class dumped her boyfriend following simons advice after he would not have sex with her but later realised this was due to him catching crabs off her friend pauline
Re-elected in the 2007 election, she was re-named the Minister of International Relations, La Francophonie and for the Estrie Region as well as the Vice-Chair

## Text Preprocessing 

### Tokenization 

In [18]:
import nltk
nltk.download('punkt') # word_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/bernatsort/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
train_df_clean['tokenized'] = train_df_clean['text_clean'].apply(word_tokenize)
# double check
print(train_df_clean["Text"][0])
print(train_df_clean["tokenized"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["tokenized"][100])
display(train_df_clean.head())

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
['zoe', 'telford', 'played', 'the', 'police', 'officer', 'girlfriend', 'of', 'simon', 'maggie', 'dumped', 'by', 'simon', 'in', 'the', 'final', 'episode', 'of', 'series', '1', 'after', 'he', 'slept', 'with', 'jenny', 'and', 'is', 'not', 'seen', 'again', 'phoebe', 'thomas', 'played', 'cheryl', 'cassidy', 'paulines', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simons', 'class', 'dumped', 'her', 'boyfriend', 'following', 'simons', 'advice', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realised', 'this', 'was', 'due', 'to',

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,A-coref,B-coref,text_clean,tokenized
0,Zoe Telford -- played the police officer girlf...,her,274,cheryl cassidy,191,pauline,207,True,False,zoe telford played the police officer girlfri...,"[zoe, telford, played, the, police, officer, g..."
1,"He grew up in Evanston, Illinois the second ol...",His,284,mackenzie,228,bernard leach,251,True,False,he grew up in evanston illinois the second old...,"[he, grew, up, in, evanston, illinois, the, se..."
2,"He had been reelected to Congress, but resigne...",his,265,angeloz,173,de la sota,246,False,True,he had been reelected to congress but resigned...,"[he, had, been, reelected, to, congress, but, ..."
3,The current members of Crime have also perform...,his,321,hell,174,henry rosenthal,336,False,True,the current members of crime have also perform...,"[the, current, members, of, crime, have, also,..."
4,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,kitty oppenheimer,219,rivera,294,False,True,her santa fe opera debut in 2005 was as nuria ...,"[her, santa, fe, opera, debut, in, 2005, was, ..."


### Stemming

Probamos los 3 tipos de stemming y luego elegimos el que haya dado mejores resultados.

#### PorterStemmer

In [20]:
from nltk.stem import PorterStemmer

In [21]:
def porter_stemmer(text):
    """
        Stem words in list of tokenized words with PorterStemmer
    """
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [22]:
train_df_clean['porter_stemmer'] = train_df_clean['tokenized'].apply(lambda x: porter_stemmer(x))

# double check
print(train_df_clean["Text"][0])
print(train_df_clean["porter_stemmer"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["porter_stemmer"][100])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
['zoe', 'telford', 'play', 'the', 'polic', 'offic', 'girlfriend', 'of', 'simon', 'maggi', 'dump', 'by', 'simon', 'in', 'the', 'final', 'episod', 'of', 'seri', '1', 'after', 'he', 'slept', 'with', 'jenni', 'and', 'is', 'not', 'seen', 'again', 'phoeb', 'thoma', 'play', 'cheryl', 'cassidi', 'paulin', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'advic', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realis', 'thi', 'wa', 'due', 'to', 'him', 'catch', 'crab', 'off

#### SnowballStemmer


In [23]:
from nltk.stem import SnowballStemmer


In [24]:
def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems

In [25]:
train_df_clean['snowball_stemmer'] = train_df_clean['tokenized'].apply(lambda x: snowball_stemmer(x))

# double check
print(train_df_clean["Text"][0])
print(train_df_clean["snowball_stemmer"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["snowball_stemmer"][100])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
['zoe', 'telford', 'play', 'the', 'polic', 'offic', 'girlfriend', 'of', 'simon', 'maggi', 'dump', 'by', 'simon', 'in', 'the', 'final', 'episod', 'of', 'seri', '1', 'after', 'he', 'slept', 'with', 'jenni', 'and', 'is', 'not', 'seen', 'again', 'phoeb', 'thoma', 'play', 'cheryl', 'cassidi', 'paulin', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'advic', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realis', 'this', 'was', 'due', 'to', 'him', 'catch', 'crab', 'o

#### LancasterStemmer 

In [26]:
from nltk.stem import LancasterStemmer

In [27]:
def lancaster_stemmer(text):
    """
        Stem words in list of tokenized words with LancasterStemmer
    """
    stemmer = nltk.LancasterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [28]:
train_df_clean['lancaster_stemmer'] = train_df_clean['tokenized'].apply(lambda x: lancaster_stemmer(x))

# double check
print(train_df_clean["Text"][0])
print(train_df_clean["lancaster_stemmer"][0])
print(train_df_clean["Text"][100])
print(train_df_clean["lancaster_stemmer"][100])
display(train_df_clean.head())

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.
['zoe', 'telford', 'play', 'the', 'pol', 'off', 'girlfriend', 'of', 'simon', 'maggy', 'dump', 'by', 'simon', 'in', 'the', 'fin', 'episod', 'of', 'sery', '1', 'aft', 'he', 'slept', 'with', 'jenny', 'and', 'is', 'not', 'seen', 'again', 'phoeb', 'thoma', 'play', 'cheryl', 'cassidy', 'paulin', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'adv', 'aft', 'he', 'would', 'not', 'hav', 'sex', 'with', 'her', 'but', 'lat', 'real', 'thi', 'was', 'due', 'to', 'him', 'catch', 'crab', 'off', 'her', 'frien

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,A-coref,B-coref,text_clean,tokenized,porter_stemmer,snowball_stemmer,lancaster_stemmer
0,Zoe Telford -- played the police officer girlf...,her,274,cheryl cassidy,191,pauline,207,True,False,zoe telford played the police officer girlfri...,"[zoe, telford, played, the, police, officer, g...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, pol, off, girlfriend..."
1,"He grew up in Evanston, Illinois the second ol...",His,284,mackenzie,228,bernard leach,251,True,False,he grew up in evanston illinois the second old...,"[he, grew, up, in, evanston, illinois, the, se...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illino, the, seco..."
2,"He had been reelected to Congress, but resigne...",his,265,angeloz,173,de la sota,246,False,True,he had been reelected to congress but resigned...,"[he, had, been, reelected, to, congress, but, ...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re..."
3,The current members of Crime have also perform...,his,321,hell,174,henry rosenthal,336,False,True,the current members of crime have also perform...,"[the, current, members, of, crime, have, also,...","[the, current, member, of, crime, have, also, ...","[the, current, member, of, crime, have, also, ...","[the, cur, memb, of, crim, hav, also, perform,..."
4,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,kitty oppenheimer,219,rivera,294,False,True,her santa fe opera debut in 2005 was as nuria ...,"[her, santa, fe, opera, debut, in, 2005, was, ...","[her, santa, fe, opera, debut, in, 2005, wa, a...","[her, santa, fe, opera, debut, in, 2005, was, ...","[her, sant, fe, oper, debut, in, 2005, was, as..."


#### Stemming techniques comparation 

In [30]:
print(train_df_clean["Text"][0])
print("\n")
print(train_df_clean["porter_stemmer"][0])
print("\n")
print(train_df_clean["snowball_stemmer"][0])
print("\n")
print(train_df_clean["lancaster_stemmer"][0])

Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.


['zoe', 'telford', 'play', 'the', 'polic', 'offic', 'girlfriend', 'of', 'simon', 'maggi', 'dump', 'by', 'simon', 'in', 'the', 'final', 'episod', 'of', 'seri', '1', 'after', 'he', 'slept', 'with', 'jenni', 'and', 'is', 'not', 'seen', 'again', 'phoeb', 'thoma', 'play', 'cheryl', 'cassidi', 'paulin', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'advic', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realis', 'thi', 'wa', 'due', 'to', 'him', 'catch', 'crab', 'o

- Según los resultados obtenidos, parece que el Porter stemmer y el Snowball stemmer producen resultados muy similares. 

- Sin embargo, el Lancaster stemmer tiende a ser más agresivo en la separación de palabras, lo que da como resultado palabras más cortas y menos reconocibles. 

- Por tanto, finalmente elegimos el Snowball stemmer, que es un poco más agresivo que el Porter stemmer pero menos agresivo que el Lancaster stemmer, y en nustro caso es el que mejor resultado ha dado. 


### Part of Speech Tagging (POS Tagging):


In [77]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/bernatsort/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [78]:
# PoS tags
train_df_clean['pos_tags'] = train_df_clean['tokenized'].apply(nltk.tag.pos_tag)

In [103]:
# double check
print("\ntokenized:")
print(train_df_clean["tokenized"][0])
print("\npos_tags:")
print(train_df_clean["pos_tags"][0])
print("\n")
print("\ntokenized:")
print(train_df_clean["tokenized"][100])
print("\npos_tags:")
print(train_df_clean["pos_tags"][100])



tokenized:
['zoe', 'telford', 'played', 'the', 'police', 'officer', 'girlfriend', 'of', 'simon', 'maggie', 'dumped', 'by', 'simon', 'in', 'the', 'final', 'episode', 'of', 'series', '1', 'after', 'he', 'slept', 'with', 'jenny', 'and', 'is', 'not', 'seen', 'again', 'phoebe', 'thomas', 'played', 'cheryl', 'cassidy', 'paulines', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simons', 'class', 'dumped', 'her', 'boyfriend', 'following', 'simons', 'advice', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realised', 'this', 'was', 'due', 'to', 'him', 'catching', 'crabs', 'off', 'her', 'friend', 'pauline']

pos_tags:
[('zoe', 'NN'), ('telford', 'NN'), ('played', 'VBD'), ('the', 'DT'), ('police', 'NN'), ('officer', 'NN'), ('girlfriend', 'NN'), ('of', 'IN'), ('simon', 'JJ'), ('maggie', 'NN'), ('dumped', 'VBN'), ('by', 'IN'), ('simon', 'NN'), ('in', 'IN'), ('the', 'DT'), ('final', 'JJ'), ('episode', 'NN'), ('of', 'IN'), ('series', 'NN'), ('1', 'CD'), ('a

- Ahora podemos utilizar estas etiquetas para aplicar la lematización.

### Lemmatization

- Dado que la lematización es un proceso más sofisticado que tiene en cuenta el contexto gramatical y semántico, es recomendable aplicarla después de la tokenización. Esto asegurará que las palabras se reduzcan a su forma base correcta y se conserven las relaciones léxicas y semánticas adecuadas en el texto.

#### With PoS

In [85]:
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/bernatsort/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Extraemos todas las etiquetas del texto: 

In [107]:
train_df_clean['pos_tags']

0       [(zoe, NN), (telford, NN), (played, VBD), (the...
1       [(he, PRP), (grew, VBD), (up, RP), (in, IN), (...
2       [(he, PRP), (had, VBD), (been, VBN), (reelecte...
3       [(the, DT), (current, JJ), (members, NNS), (of...
4       [(her, PRP$), (santa, NN), (fe, NN), (opera, N...
                              ...                        
1995    [(fayes, NNS), (third, JJ), (husband, NN), (pa...
1996    [(the, DT), (plot, NN), (of, IN), (the, DT), (...
1997    [(grant, NN), (played, VBD), (the, DT), (part,...
1998    [(the, DT), (fashion, NN), (house, NN), (speci...
1999    [(watkins, NNS), (was, VBD), (a, DT), (close, ...
Name: pos_tags, Length: 2000, dtype: object

In [109]:
def get_unique_pos_tags(df, pos_tags_col = "pos_tags"):
    # Crear una lista para almacenar todas las etiquetas
    all_tags = []

    # Iterar sobre los elementos de la columna 'pos_tags'
    for tags in df[pos_tags_col]:
        # Obtener las etiquetas de cada par (palabra, etiqueta)
        tag_list = [tag[1] for tag in tags]
        # Agregar las etiquetas a la lista general: 
            # extend() para agregar múltiples elementos a una lista
        all_tags.extend(tag_list)

    # Obtener las etiquetas únicas sin repetidos
    unique_tags = set(all_tags)

    # Imprimir las etiquetas únicas
    return unique_tags


In [111]:
# all unique tags
print(get_unique_pos_tags(train_df_clean))

{'VBZ', 'RP', 'NNP', 'POS', 'CC', 'DT', 'NN', 'VBD', 'FW', 'RBS', '$', 'VB', 'WP$', 'JJS', 'TO', 'MD', 'VBN', 'WRB', 'PRP', 'PRP$', 'RBR', 'NNS', 'WP', 'VBP', 'RB', 'PDT', 'JJ', 'IN', 'VBG', 'NNPS', 'EX', 'JJR', 'CD', 'WDT'}


Definimos una función que mapee las etiquetas POS de NLTK a las etiquetas POS de WordNet. Esto es necesario para que el lematizador pueda interpretar correctamente las etiquetas POS:

- *WordNet only contains "open-class words": nouns, verbs, adjectives, and adverbs. Thus, excluded words include determiners, prepositions, pronouns, conjunctions, and particles.*

In [129]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('POS'):
        return wordnet.NOUN
    elif tag.startswith('RB'):
        return wordnet.ADV
    elif tag.startswith('CD'):
        return wordnet.NOUN   
    else:
        return wordnet.NOUN  # Por defecto, asumimos sustantivos si la etiqueta POS no coincide con las anteriores


In [130]:
train_df_clean['wordnet_pos'] = train_df_clean['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

In [132]:
train_df_clean['wordnet_pos']

0       [(zoe, n), (telford, n), (played, v), (the, n)...
1       [(he, n), (grew, v), (up, n), (in, n), (evanst...
2       [(he, n), (had, v), (been, v), (reelected, v),...
3       [(the, n), (current, a), (members, n), (of, n)...
4       [(her, n), (santa, n), (fe, n), (opera, n), (d...
                              ...                        
1995    [(fayes, n), (third, a), (husband, n), (paul, ...
1996    [(the, n), (plot, n), (of, n), (the, n), (film...
1997    [(grant, n), (played, v), (the, n), (part, n),...
1998    [(the, n), (fashion, n), (house, n), (speciali...
1999    [(watkins, n), (was, v), (a, n), (close, a), (...
Name: wordnet_pos, Length: 2000, dtype: object

Aplicamos el word lemmatizer: 

In [113]:
from nltk.stem import WordNetLemmatizer

In [133]:
def lemmatize_word_pos(text):
    """
    Lemmatize the tokenized words (with PoS)
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemmatized_text

In [135]:
# With PoS
train_df_clean['lemmatize_word_pos'] = train_df_clean['wordnet_pos'].apply(lambda x: lemmatize_word_pos(x))

In [138]:
train_df_clean.head(3)

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,A-coref,B-coref,text_clean,tokenized,porter_stemmer,snowball_stemmer,lancaster_stemmer,lemmatize_word,pos_tags,wordnet_pos,lemmatize_word_pos
0,Zoe Telford -- played the police officer girlf...,her,274,cheryl cassidy,191,pauline,207,True,False,zoe telford played the police officer girlfri...,"[zoe, telford, played, the, police, officer, g...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, pol, off, girlfriend...","[zoe, telford, played, the, police, officer, g...","[(zoe, NN), (telford, NN), (played, VBD), (the...","[(zoe, n), (telford, n), (played, v), (the, n)...","[zoe, telford, play, the, police, officer, gir..."
1,"He grew up in Evanston, Illinois the second ol...",His,284,mackenzie,228,bernard leach,251,True,False,he grew up in evanston illinois the second old...,"[he, grew, up, in, evanston, illinois, the, se...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illino, the, seco...","[he, grew, up, in, evanston, illinois, the, se...","[(he, PRP), (grew, VBD), (up, RP), (in, IN), (...","[(he, n), (grew, v), (up, n), (in, n), (evanst...","[he, grow, up, in, evanston, illinois, the, se..."
2,"He had been reelected to Congress, but resigne...",his,265,angeloz,173,de la sota,246,False,True,he had been reelected to congress but resigned...,"[he, had, been, reelected, to, congress, but, ...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelected, to, congress, but, ...","[(he, PRP), (had, VBD), (been, VBN), (reelecte...","[(he, n), (had, v), (been, v), (reelected, v),...","[he, have, be, reelect, to, congress, but, res..."


#### Without PoS

In [114]:
def lemmatize_word(text):
    """
    Lemmatize the tokenized words (without PoS)
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_text

In [ ]:
# Without PoS
train_df_clean['lemmatize_word'] = train_df_clean['tokenized'].apply(lambda x: lemmatize_word(x))

In [48]:
# double check
display(train_df_clean.head())

,Text,Pronoun,Pronoun-offset,A,A-offset,B,B-offset,A-coref,B-coref,text_clean,tokenized,porter_stemmer,snowball_stemmer,lancaster_stemmer,lemmatize_word
0,Zoe Telford -- played the police officer girlf...,her,274,cheryl cassidy,191,pauline,207,True,False,zoe telford played the police officer girlfri...,"[zoe, telford, played, the, police, officer, g...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, polic, offic, girlfr...","[zoe, telford, play, the, pol, off, girlfriend...","[zoe, telford, played, the, police, officer, g..."
1,"He grew up in Evanston, Illinois the second ol...",His,284,mackenzie,228,bernard leach,251,True,False,he grew up in evanston illinois the second old...,"[he, grew, up, in, evanston, illinois, the, se...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illinoi, the, sec...","[he, grew, up, in, evanston, illino, the, seco...","[he, grew, up, in, evanston, illinois, the, se..."
2,"He had been reelected to Congress, but resigne...",his,265,angeloz,173,de la sota,246,False,True,he had been reelected to congress but resigned...,"[he, had, been, reelected, to, congress, but, ...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelect, to, congress, but, re...","[he, had, been, reelected, to, congress, but, ..."
3,The current members of Crime have also perform...,his,321,hell,174,henry rosenthal,336,False,True,the current members of crime have also perform...,"[the, current, members, of, crime, have, also,...","[the, current, member, of, crime, have, also, ...","[the, current, member, of, crime, have, also, ...","[the, cur, memb, of, crim, hav, also, perform,...","[the, current, member, of, crime, have, also, ..."
4,Her Santa Fe Opera debut in 2005 was as Nuria ...,She,437,kitty oppenheimer,219,rivera,294,False,True,her santa fe opera debut in 2005 was as nuria ...,"[her, santa, fe, opera, debut, in, 2005, was, ...","[her, santa, fe, opera, debut, in, 2005, wa, a...","[her, santa, fe, opera, debut, in, 2005, was, ...","[her, sant, fe, oper, debut, in, 2005, was, as...","[her, santa, fe, opera, debut, in, 2005, wa, a..."


### Lemmatization with PoS vs Lemmatization without PoS

In [139]:
# double check
print("\nOriginal:")
print(train_df_clean["Text"][0])
print("\nlemmatization PoS:")
print(train_df_clean["lemmatize_word_pos"][0])
print("\nlemmatization no PoS:")
print(train_df_clean["lemmatize_word"][0])
print("\n")
print("\nOriginal:")
print(train_df_clean["Text"][100])
print("\nlemmatization PoS:")
print(train_df_clean["lemmatize_word_pos"][100])
print("\nlemmatization no PoS:")
print(train_df_clean["lemmatize_word"][100])


Original:
Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.

lemmatization PoS:
['zoe', 'telford', 'play', 'the', 'police', 'officer', 'girlfriend', 'of', 'simon', 'maggie', 'dump', 'by', 'simon', 'in', 'the', 'final', 'episode', 'of', 'series', '1', 'after', 'he', 'sleep', 'with', 'jenny', 'and', 'be', 'not', 'see', 'again', 'phoebe', 'thomas', 'play', 'cheryl', 'cassidy', 'paulines', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'advice', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realise', 'this', 'be

- La lematización consiste en encontrar la raíz o lema de una palabra, y el lema de una palabra puede depender de su categoría gramatical.
- Al disponer de etiquetas de PoS confiables y precisas, podemos utilizarlas para mejorar la lematización y obtener resultados más precisos.
- Observamos que la lematización con PoS da mejores resultados que sin el PoS. 
- Por tanto, nos quedamos con la lematización con PoS.

### Stemming (snowball_stemmer) vs Lemmatization with PoS

In [141]:
# double check
print("\nOriginal:")
print(train_df_clean["Text"][0])
print("\nsnowball_stemmer:")
print(train_df_clean["snowball_stemmer"][0])
print("\nlemmatization PoS:")
print(train_df_clean["lemmatize_word_pos"][0])
print("\n")
print("\nOriginal:")
print(train_df_clean["Text"][100])
print("\nsnowball_stemmer:")
print(train_df_clean["snowball_stemmer"][100])
print("\nlemmatization PoS:")
print(train_df_clean["lemmatize_word_pos"][100])


Original:
Zoe Telford -- played the police officer girlfriend of Simon, Maggie. Dumped by Simon in the final episode of series 1, after he slept with Jenny, and is not seen again. Phoebe Thomas played Cheryl Cassidy, Pauline's friend and also a year 11 pupil in Simon's class. Dumped her boyfriend following Simon's advice after he wouldn't have sex with her but later realised this was due to him catching crabs off her friend Pauline.

snowball_stemmer:
['zoe', 'telford', 'play', 'the', 'polic', 'offic', 'girlfriend', 'of', 'simon', 'maggi', 'dump', 'by', 'simon', 'in', 'the', 'final', 'episod', 'of', 'seri', '1', 'after', 'he', 'slept', 'with', 'jenni', 'and', 'is', 'not', 'seen', 'again', 'phoeb', 'thoma', 'play', 'cheryl', 'cassidi', 'paulin', 'friend', 'and', 'also', 'a', 'year', '11', 'pupil', 'in', 'simon', 'class', 'dump', 'her', 'boyfriend', 'follow', 'simon', 'advic', 'after', 'he', 'would', 'not', 'have', 'sex', 'with', 'her', 'but', 'later', 'realis', 'this', 'was', 'due', 't

- Como la precisión y la coherencia lingüística son importantes en nuestro problema, la lematización puede ser más apropiada, ya que considera el contexto gramatical y semántico de las palabras. 

- Por otro lado, el stemming es un proceso más simple que busca eliminar sufijos y prefijos de las palabras para obtener una forma truncada. El stemming puede producir resultados que no corresponden a palabras reales y puede no ser tan efectivo para capturar la forma base de las palabras.

- En nuestro caso observamos que, efectivamente, la lematización con PoS funciona mejor que el stemming. 

- Por tanto, elegimos la lematización con PoS.

## Text Features Extraction



Métodos como One-Hot Encoding o Weighted Words, como Bag of Words (BoW) - Bag of n-grams, Frequency Vectors - CountVectorizer y Term Frequency-Inverse Document Frequency (TF-IDF) no capturan ni la semántica (el significado de las palabras) ni la sintáctica (posición de las palabras en el texto). Para superar estas limitaciones utilizamos los embeddings. Los embeddings captan el contexto de una palabra teniendo en cuenta las palabras vecinas en la frase y el orden de las palabras en la frase. 


### Word2Vec Embedding

Embeddings: vector based numerical representations of text.

Como tenemos un número considerable de filas (2000), usaremos embeddings, como Word2Vec. 

- Word2Vec es un encoder de tipo denso. 
- Utiliza una red neuronal
para aprender asociaciones de palabras a partir de un
texto.
- Por cada palabra, construye un vector de n dimensiones.
- En función del número de parámetros y de los datos de
entrenamiento, es capaz de capturar el significado de las
palabras y detectar sinónimos o palabras relacionadas.
- Permite calcular similitudes → cos(θ)
- Permite hacer operaciones con los vectores de
palabras.


**Ventajas:**
- Captura la posición de las palabras en el texto (sintáctica).
- Capta el significado de las palabras (semántica). 

**Limitaciones:** 
- No puede captar el significado de la palabra a partir del texto (no capta la polisemia).
- No capta las palabras del corpus que no están en el vocabulario.


In [159]:
import gensim
from gensim.models import Word2Vec
print("gensim version:", gensim.__version__)


gensim version: 4.3.1


In [408]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

8

In [409]:
# Entrenar el modelo word2vec
word2vec_model = Word2Vec(vector_size=300, 
                          window=2, 
                          min_count=20, 
                          sample=6e-5, 
                          workers=cores-1, 
                          sg=1)

In [416]:
# Building the Vocabulary Table
sentences = train_df_clean['lemmatize_word_pos']
word2vec_model.build_vocab(sentences)

- Para implementar Word2Vec, necesitaremos los textos preprocesados y tokenizados: la columna "lemmatize_word_pos"  contiene los textos lematizados y tokenizados. 
- Utilizaremos esa columna para entrenar el modelo Word2Vec.

In [417]:
# Training of the model
word2vec_model.train(sentences, 
                     total_examples=word2vec_model.corpus_count, # 2000: len(sentences)
                     epochs=30, 
                     report_delay=1)

(637587, 4264350)

- Número de palabras procesadas y el número total de tokens en tu conjunto de datos durante el entrenamiento.

Vectorización del texto: 

In [493]:
def get_sentence_vector(words, word2vec_model):
    """
    Esta función verifica si cada palabra está presente en el vocabulario 
    del modelo Word2Vec y obtiene su vector correspondiente. 
    Luego, calcula el promedio de los vectores de palabras para obtener 
    el vector representativo de la lista de palabras.
    """
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def get_average_sentence_vector(sentences, word2vec_model):
    """
    Toma una lista de oraciones y aplica la función get_sentence_vector() 
    a cada oración para obtener los vectores de palabras promedio. 
    Luego, se realiza el promedio de estos vectores de oraciones para 
    obtener un vector de oración promedio.
    """
    sentence_vectors = [get_sentence_vector(sentence) for sentence in sentences]
    if sentence_vectors:
        return np.mean(sentence_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def embed_data_with_sentence_vector(sentences, word2vec):
    """
    Recorre las oraciones de cada muestra y obtiene el vector de oración promedio 
    utilizando get_average_sentence_vector(). Solo se agrega el vector de 
    oración promedio si no es un vector de ceros.
    """
    embedded_mean = []
    
    for sentence in sentences:
        sentence_vector = get_average_sentence_vector(sentence)
        if np.any(sentence_vector):
            embedded_mean.append(sentence_vector)
    
    return embedded_mean

In [428]:
embedded_mean = embed_data_with_sentence_vector(sentences, word2vec_model)

In [430]:
len(embedded_mean)

2000

- Las dimensiones de embedded_mean coinciden con el número de filas en X_train, ya que cada elemento de embedded_mean corresponderá a una muestra en el mismo orden.

### Preprocesamiento de las otras features y creación de X_train

Combinamos embedded_mean con las features adicionales, como las columnas de desplazamiento (Pronoun-offset, A-offset y B-offset), y las características codificadas de los nombres A y B (A y B) y Pronoun.

In [439]:
# Obtener las características adicionales
pronoun_offset = np.array(train_df['Pronoun-offset'])
a_offset = np.array(train_df['A-offset'])
b_offset = np.array(train_df['B-offset'])

# Convertir las características adicionales en vectores de desplazamiento normalizados
normalized_pronoun_offset = pronoun_offset / len(embedded_mean)
normalized_a_offset = a_offset / len(embedded_mean)
normalized_b_offset = b_offset / len(embedded_mean)

# Obtener las representaciones vectoriales para Pronoun, A y B (si están disponibles)
pronoun_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in train_df['Pronoun']]
a_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in train_df['A']]
b_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in train_df['B']]

# Combinar las características en una matriz de características
X_train = np.column_stack((embedded_mean, normalized_pronoun_offset, normalized_a_offset, normalized_b_offset, a_vector, b_vector))

In [440]:
len(X_train)

2000

### Converting target to binary

- Debemos codificar las etiquetas de clase de las variables objetivo (A-coref y B-coref) para asegurar la compatibilidad con los modelos.
-  Dabido a que tenemos clases binarias, usaremos el LabelEncoder:
    - False = 0
    - True = 1


In [293]:
import sklearn
from sklearn import preprocessing

In [294]:
def encode_target(df, target_1="A-coref", target_2="B-coref"):
    le = preprocessing.LabelEncoder()
    df[target_1] = le.fit_transform(df[target_1])
    df[target_2] = le.fit_transform(df[target_2])
    return df

In [314]:
train_df = encode_target(train_df, target_1="A-coref", target_2="B-coref")

In [315]:
train_df['A-coref'].value_counts()

A-coref
0    1126
1     874
Name: count, dtype: int64

In [316]:
train_df['B-coref'].value_counts()

B-coref
0    1075
1     925
Name: count, dtype: int64

### Añadir 3a condición: A-coref = False y B-coref = False

Puede haber casos en los que la respuesta sea A-coref == False y B-coref == False, lo que indica que el pronombre no hace referencia a ninguno de los nombres propuestos. 

In [330]:
if (~(train_df['A-coref'] | train_df['B-coref'])).any():
    print("Existe al menos una fila donde A-coref y B-coref son ambos 0.")
else:
    print("No existe ninguna fila donde A-coref y B-coref sean ambos 0.")

Existe al menos una fila donde A-coref y B-coref son ambos 0.


Por otro lado, no puede haber casos en los que la respuesta sea A-coref == True y B-coref == True:

In [331]:
if (train_df['A-coref'] & train_df['B-coref']).any():
    print("Existe al menos una fila donde A-coref y B-coref son ambos 1.")
else:
    print("No existe ninguna fila donde A-coref y B-coref sean ambos 1.")

No existe ninguna fila donde A-coref y B-coref sean ambos 1.


In [318]:
# Obtener las columnas A-coref y B-coref del dataset de entrenamiento
a_coref = train_df['A-coref'].values
b_coref = train_df['B-coref'].values

# Crear la tercera columna para representar la clase "None"
none_coref = np.where((a_coref == 0) & (b_coref == 0), 1, 0)

# Agregar la columna "None-coref" al dataset de entrenamiento
train_df['None-coref'] = none_coref


In [320]:
train_df['None-coref'].value_counts()

None-coref
0    1799
1     201
Name: count, dtype: int64

- Hay 201 casos en los que el pronombre no hace referencia a ninguno de los nombres propuestos.

In [324]:
# casos en los que el pronombre no hace referencia a ninguno de los nombres propuestos
train_df[train_df['None-coref']==1][['A-coref','B-coref','None-coref']]

,A-coref,B-coref,None-coref
6,0,0,1
15,0,0,1
17,0,0,1
28,0,0,1
31,0,0,1
...,...,...,...
1961,0,0,1
1983,0,0,1
1985,0,0,1
1986,0,0,1


In [321]:
train_df[['A-coref','B-coref','None-coref']]

,A-coref,B-coref,None-coref
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,0,1,0
...,...,...,...
1995,0,1,0
1996,0,1,0
1997,1,0,0
1998,1,0,0


- Ahora tenemos 3 variables objetivo: "A-coref", "B-coref" y "None-coref". Cada instancia tiene un valor de 1 en una de las columnas correspondientes a la variable objetivo y un valor de 0 en las otras dos columnas.



### Crear columna target 

- Creamos la columna target definitiva donde asignamos:
    - `0`: si el pronombre hace referencia al nombre A.
    - `1`: si el pronombre hace referencia al nombre B.
    - `2`: si el pronombre no hace referencia a ninguno de los dos nombres propuestos. 

In [332]:
# Crear la columna target
def get_target(row):
    if row['A-coref'] == 1:
        return 0
    elif row['B-coref'] == 1:
        return 1
    else:
        return 2

In [ ]:
# Creamos la columna target
train_df['target'] = train_df.apply(get_target, axis=1)

In [333]:
train_df[['A-coref','B-coref','None-coref', 'target']]

,A-coref,B-coref,None-coref,target
0,1,0,0,0
1,1,0,0,0
2,0,1,0,1
3,0,1,0,1
4,0,1,0,1
...,...,...,...,...
1995,0,1,0,1
1996,0,1,0,1
1997,1,0,0,0
1998,1,0,0,0


In [334]:
train_df[train_df['None-coref']==1][['A-coref','B-coref','None-coref', 'target']]

,A-coref,B-coref,None-coref,target
6,0,0,1,2
15,0,0,1,2
17,0,0,1,2
28,0,0,1,2
31,0,0,1,2
...,...,...,...,...
1961,0,0,1,2
1983,0,0,1,2
1985,0,0,1,2
1986,0,0,1,2


In [337]:
# 3 clases
train_df['target'].value_counts()

target
1    925
0    874
2    201
Name: count, dtype: int64

- 874 instancias donde el pronombre hace referencia al nombre A.
- 925 instancias donde el pronombre hace referencia al nombre B.
- 201 instancias donde el pronombre no hace referencia a ninguno de los dos nombres propuestos.

## Full preprocessing pipeline

Encapsulamos todos los pasos de limpieza y preprocesamiento anteriores en funciones. 
De esta manera, podremos aplicar todo el preprocesamiento de los datos al train set, validation set y test set por separado y de una manera más eficaz. 

In [474]:
# Text Cleaning Functions

def select_imp_features(df):
    imp_features = ["Text", "Pronoun", "Pronoun-offset", "A", "A-offset", "B", "B-offset"]
    target_col = ["A-coref", "B-coref"]
    df = df[imp_features + target_col]
    return df 

def lower_case(df):
    df.loc[:, "text_clean"] = df["Text"].apply(lambda x: x.lower())
    df.loc[:, "Pronoun"] = df["Pronoun"].apply(lambda x: x.lower())
    df.loc[:, "A"] = df["A"].apply(lambda x: x.lower())
    df.loc[:, "B"] = df["B"].apply(lambda x: x.lower())

def expand_contractions(df):
    df.loc[:, "text_clean"] = df["text_clean"].apply(lambda x: contractions.fix(x))

def remove_non_ascii_characters(df, col='text_clean'):
    df.loc[:, col] = df[col].apply(lambda text: re.sub(r'[^\x00-\x7f]', r'', text)) # get rid of non-characters and whitespace
    return df

def remove_punctuations(df, col='text_clean'):
    """
     - str.maketrans('', '', string.punctuation) crea un traductor utilizando maketrans 
       que mapea los caracteres de puntuación a None, es decir, los elimina.
     - string.punctuation es una cadena predefinida en el módulo string que contiene todos 
       los caracteres de puntuación.
     - text.translate(translator) aplica el traductor al texto, reemplazando las puntuaciones 
       con caracteres vacíos, lo que efectivamente las elimina.
    """
    df.loc[:, col] = df[col].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))
    # return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
    return df

# Orquestación secuencial de la limpieza de texto
def text_cleaning(df):
    # Crear una copia del DataFrame:
        # todas las modificaciones se realicen en una copia independiente 
        # y no afectan al DataFrame original.
    df = df.copy()  
    # text cleaning functions
    df = select_imp_features(df)
    lower_case(df)
    expand_contractions(df)
    df = remove_non_ascii_characters(df)
    df = remove_punctuations(df)
    return df

In [480]:
# Text Preprocessing Functions 

# Tokenization
def tokenize_text(df):
    df.loc[:, 'tokenized'] = df['text_clean'].apply(word_tokenize)

# Part of Speech Tagging
def part_of_speech_tagging(df):
    df.loc[:, 'pos_tags'] = df['tokenized'].apply(nltk.tag.pos_tag)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('POS'):
        return wordnet.NOUN
    elif tag.startswith('RB'):
        return wordnet.ADV
    elif tag.startswith('CD'):
        return wordnet.NOUN   
    else:
        return wordnet.NOUN  # Por defecto, asumimos sustantivos si la etiqueta POS no coincide con las anteriores

def apply_wordnet_pos(df):
    df.loc[:, 'wordnet_pos'] = df['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
    
# Lemmatization with PoS
def lemmatize_word_pos(text):
    """
    Lemmatize the tokenized words (with PoS)
    """
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemmatized_text

def apply_lemmatize_word_pos(df):
    df.loc[:, 'lemmatize_word_pos'] = df['wordnet_pos'].apply(lambda x: lemmatize_word_pos(x))

# Orquestación secuencial del preprocesamiento del texto
def text_preprocessing(df):
    df = df.copy()

    tokenize_text(df)
    part_of_speech_tagging(df)
    apply_wordnet_pos(df)
    apply_lemmatize_word_pos(df)

    return df

In [496]:
# Text Features Extraction

# Word2Vec Embedding
def train_word2vec_model(sentences):
    # Set up the parameters of the model
    word2vec_model = Word2Vec(vector_size=300, 
                              window=2, 
                              min_count=20, 
                              sample=6e-5, 
                              workers=cores-1, 
                              sg=1)
                              
    # Building the Vocabulary Table
    word2vec_model.build_vocab(sentences)
    
    # Training the model
    word2vec_model.train(sentences, 
                         total_examples=word2vec_model.corpus_count,  # 2000: len(sentences)
                         epochs=30, 
                         report_delay=1)
    
    return word2vec_model

# Text Vectorization
def get_sentence_vector(words, word2vec_model):
    """
    Esta función verifica si cada palabra está presente en el vocabulario 
    del modelo Word2Vec y obtiene su vector correspondiente. 
    Luego, calcula el promedio de los vectores de palabras para obtener 
    el vector representativo de la lista de palabras.
    """
    word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
    if word_vectors:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def get_average_sentence_vector(sentences, word2vec_model):
    """
    Toma una lista de oraciones y aplica la función get_sentence_vector() 
    a cada oración para obtener los vectores de palabras promedio. 
    Luego, se realiza el promedio de estos vectores de oraciones para 
    obtener un vector de oración promedio
    """
    sentence_vectors = [get_sentence_vector(sentence, word2vec_model) for sentence in sentences]
    if sentence_vectors:
        return np.mean(sentence_vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

def embed_data_with_sentence_vector(sentences, word2vec_model):
    """
    Recorre las oraciones de cada muestra y obtiene el vector de oración promedio 
    utilizando get_average_sentence_vector(). Solo se agrega el vector de 
    oración promedio si no es un vector de ceros.
    """
    embedded_mean = []
    
    for sentence in sentences:
        sentence_vector = get_average_sentence_vector(sentence, word2vec_model)
        if np.any(sentence_vector):
            embedded_mean.append(sentence_vector)
    
    return embedded_mean

def word2vec_text_vectorization_embed(df):
    df = df.copy()

    # Entrenar el modelo Word2Vec
    sentences = df['lemmatize_word_pos']
    word2vec_model = train_word2vec_model(sentences)
    # Obtener los vectores promedio de las frases
    embedded_mean = embed_data_with_sentence_vector(sentences, word2vec_model)
    
    return word2vec_model, embedded_mean

In [500]:
# Preprocessing the other features
def get_additional_features(df, word2vec_model, embedded_mean):
    """
    Combinamos embedded_mean con las features adicionales, como las columnas 
    de desplazamiento (Pronoun-offset, A-offset y B-offset), 
    y las características codificadas de los nombres A y B (A y B) y Pronoun.
    """
    df = df.copy()
    
    # Obtener las características adicionales
    pronoun_offset = np.array(df['Pronoun-offset'])
    a_offset = np.array(df['A-offset'])
    b_offset = np.array(df['B-offset'])

    # Convertir las características adicionales en vectores de desplazamiento normalizados
    normalized_pronoun_offset = pronoun_offset / len(embedded_mean)
    normalized_a_offset = a_offset / len(embedded_mean)
    normalized_b_offset = b_offset / len(embedded_mean)

    # Obtener las representaciones vectoriales para Pronoun, A y B (si están disponibles)
    pronoun_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in df['Pronoun']]
    a_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in df['A']]
    b_vector = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(word2vec_model.vector_size) for word in df['B']]

    # Combinar las características en una matriz de características
    X = np.column_stack((embedded_mean, normalized_pronoun_offset, normalized_a_offset, normalized_b_offset, a_vector, b_vector))
    
    return X

In [505]:
# Converting target to binary
def encode_target(df, target_1="A-coref", target_2="B-coref"):
    """
    Debemos codificar las etiquetas de clase de las variables objetivo 
    (A-coref y B-coref) para asegurar la compatibilidad con los modelos.
    Dabido a que tenemos clases binarias, usaremos el LabelEncoder:
        - False = 0
        - True = 1
    """
    le = preprocessing.LabelEncoder()
    df[target_1] = le.fit_transform(df[target_1])
    df[target_2] = le.fit_transform(df[target_2])

    return df

In [516]:
# CREAR COLUMNA TARGET DEFINITIVA
# Añadir 3a condición: pronombre no hace referencia a ninguno de los nombres propuestos
def add_none_coref_column(df):
    """
    Puede haber casos en los que la respuesta sea A-coref == False y B-coref == False, 
    lo que indica que el pronombre no hace referencia a ninguno de los nombres propuestos. 
    """
    # Obtener las columnas A-coref y B-coref del dataset de entrenamiento
    a_coref = df['A-coref'].values
    b_coref = df['B-coref'].values

    # Crear la tercera columna para representar la clase "None"
    none_coref = np.where((a_coref == 0) & (b_coref == 0), 1, 0)

    # Agregar la columna "None-coref" al dataset de entrenamiento
    df['None-coref'] = none_coref

    return df

# Crear la columna target
def get_target(row):
    """
    Creamos la columna target definitiva donde asignamos:
    - `0`: si el pronombre hace referencia al nombre A.
    - `1`: si el pronombre hace referencia al nombre B.
    - `2`: si el pronombre no hace referencia a ninguno de los dos nombres propuestos. 
    """
    if row['A-coref'] == 1:
        return 0
    elif row['B-coref'] == 1:
        return 1
    else:
        return 2

def multiclass_target(df):
    """
    Crea la columna target definitiva.
    Agrega la columna "None-coref" y "target" al df, 
    donde la columna "target" contendrá los valores 0, 1 o 2 
    dependiendo de si el pronombre hace referencia al nombre A, B 
    o a ninguno de los dos nombres propuestos.
    """
    df = df.copy()
    # Añadir la columna None-coref al df
    df = add_none_coref_column(df)
    # Crear la columna target 
    df['target'] = df.apply(get_target, axis=1)

    return df

In [548]:
def preprocess_dataset(df):
    """
    Realiza el preprocesamiento completo del dataset.
    Devuelve las características X y los valores objetivo y.
    """
    
    # Text Cleaning
    df_cleaned = text_cleaning(df)

    # Text Preprocessing
    df_clean_prep = text_preprocessing(df_cleaned)

    # Text Features Extraction
    word2vec_model, embedded_mean = word2vec_text_vectorization_embed(df_clean_prep)

    # Preprocessing the other features
    # Build the X (features)
    X = get_additional_features(df_clean_prep, 
                                word2vec_model,
                                embedded_mean)

    # Converting target to binary
    df_clean_prep = encode_target(df_clean_prep, 
                                  target_1="A-coref", 
                                  target_2="B-coref")

    # Creamos la columna target definitiva
    df_final = multiclass_target(df_clean_prep)

    # y (target)
    y = df_final['target'].values.astype(int)
    
    return X, y

### Train set preprocessing

In [ ]:
# Load train data
DATA_ROOT = './input/'
GAP_DATA_FOLDER = os.path.join(DATA_ROOT, 'gap-coreference-master')
train_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-development.tsv')
train_df = pd.read_csv(train_df_path, sep='\t') 

# Preprocess the TRAIN set
X_train, y_train = preprocess_dataset(train_df)

### Validation set preprocessing

In [560]:
# Load validation data
DATA_ROOT = './input/'
GAP_DATA_FOLDER = os.path.join(DATA_ROOT, 'gap-coreference-master')
val_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-validation.tsv')
val_df = pd.read_csv(val_df_path, sep='\t')

# Preprocess the VALIDATION set
X_val, y_val = preprocess_dataset(val_df)

### Test set Preprocessing

In [556]:
# Load test data
DATA_ROOT = './input/'
GAP_DATA_FOLDER = os.path.join(DATA_ROOT, 'gap-coreference-master')
test_df_path = os.path.join(GAP_DATA_FOLDER, 'gap-test.tsv')
test_df = pd.read_csv(test_df_path, sep='\t')

# Preprocess the TEST set
X_test, y_test = preprocess_dataset(test_df)


## Model 1

- Text classification with 3 different classes.
- Evaluation metric: multi-class log-loss.
    -  La métrica de accuracy es ampliamente utilizada y fácil de interpretar, ya que representa la proporción de predicciones correctas en relación con el total de predicciones. Sin embargo, la accuracy puede ser engañosa en casos donde las clases están desbalanceadas, es decir, cuando algunas clases tienen muchos más ejemplos que otras. En estos casos, el modelo puede tener una alta accuracy al predecir la clase mayoritaria, pero tener un rendimiento deficiente en la predicción de las clases minoritarias.

    - El log loss (pérdida logarítmica) es una métrica más adecuada cuando se trata de problemas de clasificación multiclase con clases desbalanceadas, como es nuestro caso. El log loss penaliza las predicciones incorrectas de manera más fuerte, y su valor se minimiza cuando las probabilidades asignadas a las clases correctas son cercanas a 1.0. Es una métrica más sensible a los errores de clasificación y proporciona una medida más precisa de la calidad de las predicciones en un problema multiclase.

Recordemos: 
- Clase 0: 874 instancias donde el pronombre hace referencia al nombre A.
- Clase 1: 925 instancias donde el pronombre hace referencia al nombre B.
- Clase 2: 201 instancias donde el pronombre no hace referencia a ninguno de los dos nombres propuestos.

Las clases no están perfectamente equilibradas, pero tampoco existe un desequilibrio extremo.

Dado que la clase "0" y la clase "1" tienen una cantidad similar de instancias, mientras que la clase "2" tiene menos instancias, es probable que el rendimiento del modelo en la clasificación de las clases "0" y "1" sea más representativo de su capacidad general. Sin embargo, esto no significa que la clase "2" deba ser descartada o ignorada, ya que sigue siendo una clase importante de nuestro problema.

Al utilizar el log loss como métrica de evaluación, el modelo será penalizado de manera más significativa si no puede asignar una alta probabilidad a la clase correcta, tanto en las clases "0" y "1" como en la clase "2". Esto asegurará que el modelo sea más consciente de la importancia de todas las clases y buscará mejorar su rendimiento en cada una de ellas.

Por tanto, al utilizar el log loss como métrica de evaluación, estaremos evaluando el rendimiento del modelo de manera justa y equilibrada en todas las clases, incluida la clase "2" que tiene menos instancias. Esto nos permitirá obtener una evaluación global más precisa del modelo y tomar decisiones informadas sobre su rendimiento.

In [590]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [577]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

### Logistic Regression

In [593]:
# # Logistic Regression
# clf = LogisticRegression(C=1.0)
# clf.fit(X_train, y_train)
# predictions = clf.predict_proba(X_val)

# print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

# # Convertir probabilidades a etiquetas
# predicted_labels = np.argmax(predictions, axis=1)  

# # Imprimir el informe de clasificación
# print(classification_report(y_val, predicted_labels, zero_division=1))

logloss: 1.387 
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       187
           1       0.45      1.00      0.62       205
           2       1.00      0.00      0.00        62

    accuracy                           0.45       454
   macro avg       0.82      0.33      0.21       454
weighted avg       0.75      0.45      0.28       454



#### Hyperparameter Tuning: Grid Search

In [609]:
# Before starting with grid search we need to create a scoring function. 
# This is accomplished using the make_scorer function of scikit-learn.

mll_scorer = metrics.make_scorer(multiclass_logloss, 
                                 greater_is_better=False, # el log loss debe ser minimizado
                                 needs_proba=True)

In [610]:
# Logistic Regression
log_reg = LogisticRegression()

# grid of parameters: evaluate three different values of C with l1 and l2 penalty.
# param_grid = {'C': [0.01, 1.0, 10], 
#               'penalty': ['none', 'l2']}

param_grid  = [
                {'solver' : ['saga'],
                'penalty' : ['elasticnet', 'l1', 'l2', 'none'],
                'max_iter' : [50,100,200,500,1000,2500],

                'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
                {'solver' : ['newton-cg', 'lbfgs'],
                'penalty' : ['l2','none'],
                'max_iter' : [50,100,200,500,1000,2500],
                'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
              ]

In [611]:
# We can now start grid search on these parameters
# Initialize Grid Search Model
model = GridSearchCV(estimator=log_reg, 
                     param_grid=param_grid, 
                     scoring=mll_scorer,
                     verbose=10, 
                     n_jobs=-1, # using all available cores
                     refit=True, # el modelo final se ajustará con 
                                 # los mejores hiperparámetros encontrados después de la búsqueda.
                     cv=2)

# Fit Grid Search Model
model.fit(X_train, y_train)  # we can use the full data here but im only using xtrain
print("Best score: %0.3f" % model.best_score_)

print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for params in param_grid:
    for param_name in sorted(params.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 336 candidates, totalling 672 fits


340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.13s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
340601.14s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


[CV 2/2; 2/336] START C=0.001, max_iter=50, penalty=l1, solver=saga.............
[CV 2/2; 4/336] START C=0.001, max_iter=50, penalty=none, solver=saga...........
[CV 2/2; 3/336] START C=0.001, max_iter=50, penalty=l2, solver=saga.............
[CV 1/2; 3/336] START C=0.001, max_iter=50, penalty=l2, solver=saga.............
[CV 1/2; 2/336] START C=0.001, max_iter=50, penalty=l1, solver=saga.............
[CV 1/2; 4/336] START C=0.001, max_iter=50, penalty=none, solver=saga...........
[CV 2/2; 1/336] START C=0.001, max_iter=50, penalty=elasticnet, solver=saga.....
[CV 1/2; 1/336] START C=0.001, max_iter=50, penalty=elasticnet, solver=saga.....
[CV 2/2; 1/336] END C=0.001, max_iter=50, penalty=elasticnet, solver=saga;, score=nan total time=   0.0s
[CV 1/2; 1/336] END C=0.001, max_iter=50, penalty=elasticnet, solver=saga;, score=nan total time=   0.0s
[CV 1/2; 2/336] END C=0.001, max_iter=50, penalty=l1, solver=saga;, score=-0.953 total time=   0.1s
[CV 2/2; 2/336] END C=0.001, max_iter=50, 

/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1181: UserWarni

[CV 1/2; 3/336] END C=0.001, max_iter=50, penalty=l2, solver=saga;, score=-0.949 total time=   0.4s
[CV 1/2; 7/336] END C=0.001, max_iter=100, penalty=l2, solver=saga;, score=-0.949 total time=   0.3s
[CV 1/2; 9/336] START C=0.001, max_iter=200, penalty=elasticnet, solver=saga....
[CV 2/2; 9/336] START C=0.001, max_iter=200, penalty=elasticnet, solver=saga....
[CV 1/2; 9/336] END C=0.001, max_iter=200, penalty=elasticnet, solver=saga;, score=nan total time=   0.0s
[CV 1/2; 10/336] START C=0.001, max_iter=200, penalty=l1, solver=saga...........
[CV 2/2; 9/336] END C=0.001, max_iter=200, penalty=elasticnet, solver=saga;, score=nan total time=   0.0s
[CV 2/2; 10/336] START C=0.001, max_iter=200, penalty=l1, solver=saga...........
[CV 2/2; 7/336] END C=0.001, max_iter=100, penalty=l2, solver=saga;, score=-0.950 total time=   0.3s
[CV 1/2; 11/336] START C=0.001, max_iter=200, penalty=l2, solver=saga...........
[CV 2/2; 10/336] END C=0.001, max_iter=200, penalty=l1, solver=saga;, score=-0.99

...

/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/2; 336/336] END C=1000, max_iter=2500, penalty=none, solver=lbfgs;, score=-1.355 total time=   6.1s


/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
84 fits failed out of a total of 672.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the 

[CV 2/2; 336/336] END C=1000, max_iter=2500, penalty=none, solver=lbfgs;, score=-1.867 total time=   5.7s
Best score: -0.946
Best parameters set:
	C: 1
	max_iter: 50
	penalty: 'l1'
	solver: 'saga'
	C: 1
	max_iter: 50
	penalty: 'l1'
	solver: 'saga'


/Users/bernatsort/Library/CloudStorage/OneDrive-LaSalle/1. MASTER_DS/NLP/nlp_env/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


- Best score: -0.946
- Best parameters set:
	- C: 1
	- max_iter: 50
	- penalty: 'l1'
	- solver: 'saga'
	- C: 1
	- max_iter: 50
	- penalty: 'l1'
	- solver: 'saga'




### SVC

In [565]:
from sklearn.svm import SVC
from sklearn import decomposition

# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(X_train)
xtrain_svd = svd.transform(X_train)
xvalid_svd = svd.transform(X_val)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.
scl = preprocessing.StandardScaler()
scl.fit(X_train)
xtrain_svd_scl = scl.transform(X_train)
xvalid_svd_scl = scl.transform(X_val)

In [575]:
# Fitting a simple SVM
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, y_train)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

# Convertir probabilidades a etiquetas
predicted_labels = np.argmax(predictions, axis=1)  

# Imprimir el informe de clasificación
print(classification_report(y_val, predicted_labels, zero_division=1))

logloss: 1.001 
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       187
           1       0.45      1.00      0.62       205
           2       1.00      0.00      0.00        62

    accuracy                           0.45       454
   macro avg       0.82      0.33      0.21       454
weighted avg       0.75      0.45      0.28       454



### XGBoost

In [583]:
import xgboost as xgb
# Fitting a simple xgboost 
clf = xgb.XGBClassifier(max_depth=7,
                        n_estimators=200, 
                        colsample_bytree=0.8, 
                        subsample=0.8, 
                        nthread=10, 
                        learning_rate=0.1)

clf.fit(X_train, y_train)
predictions = clf.predict_proba(X_val)

print ("logloss: %0.3f " % multiclass_logloss(y_val, predictions))

# Convertir probabilidades a etiquetas
predicted_labels = np.argmax(predictions, axis=1)  

# Imprimir el informe de clasificación
print(classification_report(y_val, predicted_labels, zero_division=1))

logloss: 1.049 
              precision    recall  f1-score   support

           0       0.50      0.01      0.01       187
           1       0.45      1.00      0.62       205
           2       1.00      0.00      0.00        62

    accuracy                           0.45       454
   macro avg       0.65      0.33      0.21       454
weighted avg       0.55      0.45      0.28       454

